In [1]:
!pip install spacy
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install tqdm
!pip install datasets
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because norm

In [1]:
# Import necessary libraries
from transformers import pipeline, AutoTokenizer
import spacy
import pandas as pd
from tqdm import tqdm
import requests
from functools import lru_cache


# Load the spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Function to load relation schema from a CSV file
def load_relation_schema(csv_file):
    # Load CSV into DataFrame
    df = pd.read_csv(csv_file)
    # Convert 'domains' and 'ranges' columns to strings, replace NaN with empty string
    df['domains'] = df['domains'].fillna('').astype(str)
    df['ranges'] = df['ranges'].fillna('').astype(str)
    # Extract relevant columns and convert to list of tuples
    relations = [(row['property'], row['propertyLabel'], row['propertyDescription'], row['domains'], row['ranges']) for _, row in df.iterrows()]
    return relations

# Load the relation schema from a CSV file
relation_schema = load_relation_schema('data/relations.csv')

# Load the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.2')

# Check and set the PAD token for the tokenizer
pad_token = tokenizer.pad_token
if pad_token is None:
    tokenizer.pad_token = tokenizer.unk_token

# Initialize the Hugging Face pipeline with a generative model
generator = pipeline('text-generation', model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer=tokenizer, device_map="auto", model_kwargs={"load_in_8bit": True})



#Checks weather a class is a subclass of another class. This is required for the range/domain check.
@lru_cache(maxsize=None)
def query_subclass_of(subclass_id, class_id):
    """
    Check if the given subclass_id is a subclass of class_id using Wikidata SPARQL endpoint.
    Uses caching to reduce duplicate queries.
    """
    sparql_query = f"""
    ASK {{
        wd:{subclass_id} (wdt:P279)* wd:{class_id} .
    }}
    """
    response = query_wikidata_sparql(sparql_query)
    return response.get('boolean', False)


#Checks type compatibility. This is deactivated at the moment.
def is_compatible(entity1_classes, entity2_classes, relation_domain, relation_range):
    """
    Check if entity1's classes are compatible with relation_domain and
    entity2's classes with relation_range, or vice versa.
    """
    # Assuming query_subclass_of function is defined to check subclass relationships
    # and direct class matches.

    domain_classes = set(relation_domain.split(', '))
    range_classes = set(relation_range.split(', '))

    # Check compatibility for entity1 with domain and entity2 with range
    compatibility1 = any(query_subclass_of(cls[0], domain[0]) for cls in entity1_classes for domain in domain_classes) \
                     and any(query_subclass_of(cls[0], rng[0]) for cls in entity2_classes for rng in range_classes)

    # Check compatibility for entity2 with domain and entity1 with range
    compatibility2 = any(query_subclass_of(cls[0], domain[0]) for cls in entity2_classes for domain in domain_classes) \
                     and any(query_subclass_of(cls[0], rng[0]) for cls in entity1_classes for rng in range_classes)

    return compatibility1 or compatibility2


# Function to generate prompts for relation extraction
def generate_prompts(candidate_sentences, relation_schema, linked_entities_dict):
    prompts = []
    entity_info = []
    # Iterate through each candidate sentence to generate prompts
    for sentence in candidate_sentences:
        entities_in_sentence = [ent.text for ent in sentence.ents if ent.text in linked_entities_dict]
        if len(entities_in_sentence) > 1:
            for i in range(len(entities_in_sentence)):
                for j in range(i + 1, len(entities_in_sentence)):
                    entity1_classes = linked_entities_dict[entities_in_sentence[i]]
                    entity2_classes = linked_entities_dict[entities_in_sentence[j]]
                    # Check compatibility and generate prompt if compatible
                    # Inside the generate_prompts function loop
                    for _, label, description, relation_domain, relation_range in relation_schema:
                        # Adjusted to pass entity classes directly corresponding to domain and range
                        #if is_compatible(entity1_classes, entity2_classes, relation_domain, relation_range):
                        # Generate prompt if compatible
                        prompt = f"[INST]Answer with yes or no. Does the sentence '{sentence.text}' contain the relationship '{label}' between '{entities_in_sentence[i]}' and '{entities_in_sentence[j]}'? Only answer with yes, if you are sure[/INST]"
                        prompts.append(prompt)
                        entity_info.append((entities_in_sentence[i], entities_in_sentence[j], description))
    return prompts, entity_info

# Function to get entity mentions from a sentence
def get_entity_mentions(sentence):
    doc = nlp(sentence)
    return [ent.text for ent in doc.ents]

# Function to query Wikidata SPARQL endpoint
def query_wikidata_sparql(query):
    url = "https://query.wikidata.org/sparql"
    headers = {'User-Agent': 'TypeCheck; https://enexa.eu/contact/'}

    response = requests.get(url,headers=headers, params={'format': 'json', 'query': query})
    print(response)
    return response.json()

# Function to search for an entity in Wikidata and get its classes
def search_wikidata(entity):
    search_url = "https://www.wikidata.org/w/api.php"
    search_params = {
        "action": "wbsearchentities",
        "language": "en",
        "format": "json",
        "search": entity
    }
    search_response = requests.get(search_url, params=search_params)
    search_results = search_response.json().get("search", [])
    if search_results:
        wikidata_id = search_results[0].get("id")  # Get the ID of the first match
        # Query SPARQL to get all classes for this entity
        sparql_query = f"""
        SELECT ?class ?classLabel WHERE {{
            wd:{wikidata_id} wdt:P31 ?class .
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
        results = query_wikidata_sparql(sparql_query)
        # Extract class IDs and labels from query results
        classes = [(result['class']['value'].split('/')[-1], result['classLabel']['value']) for result in results['results']['bindings']]
        return wikidata_id, classes
    return None, []

# Function to link entities in sentences to Wikidata
def link_entities_to_wikidata(candidate_sentences):
    linked_entities = []
    # Iterate through candidate sentences to link entities
    for sentence in candidate_sentences:
        sentence_text = sentence.text  # Convert the span to a string
        entity_mentions = get_entity_mentions(sentence_text)
        for entity in entity_mentions:
            wikidata_id, classes = search_wikidata(entity)
            if wikidata_id:
                linked_entities.append((entity, wikidata_id, classes))
    return linked_entities

# Function to extract relations in batch using the generator
def batch_extract_relations(candidate_sentences, relation_schema, generator, linked_entities_dict, batch_size=512):
    extracted_relations = []
    prompts, entity_info_list = generate_prompts(candidate_sentences, relation_schema, linked_entities_dict)
    
    prompt_generator = (prompt for prompt in prompts)  # Create generator from prompts list

    # Iterate through generator responses and process them
    for i, response in enumerate(tqdm(generator(prompt_generator, max_new_tokens=4, batch_size=batch_size), desc="Processing")):
        generated_text = response[0]['generated_text']
        subject, object, relation_description = entity_info_list[i]
        # Check if the generated text indicates a "yes" response
        if "yes" in generated_text.lower():
            # Find the relation label corresponding to the description
            relation_label = next((label for _, label, description, _, _ in relation_schema if description == relation_description), None)
            if relation_label:
                extracted_relations.append((subject, relation_label, object))

    return extracted_relations


# Process input text with spaCy to identify candidate sentences for relation extraction
text = """
Berlin is located in Germany.
Barack Obama was born in Honolulu.
"""
doc = nlp(text)
candidate_sentences = [sent for sent in doc.sents if len(sent.ents) >= 2]
print("Candidated Sentences are:")
print(candidate_sentences)
# Link entities to Wikidata for further processing
linked_entities = link_entities_to_wikidata(candidate_sentences)
print(linked_entities)
linked_entities_dict = {entity: classes for entity, _, classes in linked_entities}

# Extract relations for all candidate sentences in a batch
extracted_relations = batch_extract_relations(candidate_sentences, relation_schema, generator, linked_entities_dict, batch_size=512)

# Print extracted relations
print("Extracted Relations:")
for relation in extracted_relations:
    print(relation)



2024-01-31 11:53:41.364279: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 11:53:41.593983: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Candidated Sentences are:
[
Berlin is located in Germany.
, Barack Obama was born in Honolulu.
]
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
[('Berlin', 'Q64', [('Q5119', 'capital city'), ('Q133442', 'city-state'), ('Q200250', 'metropolis'), ('Q707813', 'Hanseatic city'), ('Q1221156', 'federated state of Germany'), ('Q1549591', 'big city'), ('Q1637706', 'million city'), ('Q1901835', 'seat of government'), ('Q15974307', 'Einheitsgemeinde of Germany'), ('Q42744322', 'urban municipality in Germany'), ('Q51929311', 'largest city'), ('Q67123843', 'town divided by border'), ('Q114401982', 'independent city in Berlin')]), ('Germany', 'Q183', [('Q6256', 'country'), ('Q3624078', 'sovereign state')]), ('Barack Obama', 'Q76', [('Q5', 'human')]), ('Honolulu', 'Q18094', [('Q62049', 'county seat'), ('Q1549591', 'big city'), ('Q21518270', 'state or insular area capital of the United States')])]


Processing: 0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing: 372it [00:07, 86.94it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing: 1017it [00:14, 143.79it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing: 1482it [00:21, 110.87it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing: 1968it [00:28, 107.61it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing: 2425it [00:35, 94.41it/s] Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing: 3023it [00:43, 127.96it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing: 3244it [00:46, 70.29it/s] 

Extracted Relations:
('Berlin', 'lyrics by', 'Germany')
('Berlin', 'ex taxon author', 'Germany')
('Berlin', 'voice actor', 'Germany')
('Berlin', 'residence', 'Germany')
('Berlin', 'director of photography', 'Germany')
('Berlin', 'presenter', 'Germany')
('Berlin', 'designed by', 'Germany')
('Berlin', 'head coach', 'Germany')
('Berlin', 'team captain', 'Germany')
('Berlin', 'translator', 'Germany')
('Berlin', 'organizer', 'Germany')
('Berlin', 'commander of (DEPRECATED)', 'Germany')
('Berlin', 'encoded by', 'Germany')
('Berlin', 'located on astronomical body', 'Germany')
('Berlin', 'location', 'Germany')
('Berlin', 'central bank/issuer', 'Germany')
('Berlin', 'terminus', 'Germany')
('Berlin', 'parent astronomical body', 'Germany')
('Berlin', 'companion of', 'Germany')
('Berlin', 'child astronomical body', 'Germany')
('Berlin', 'place of publication', 'Germany')
('Berlin', 'replaced synonym (for nom. nov.)', 'Germany')
('Berlin', 'found in taxon', 'Germany')
('Berlin', 'sport', 'Germany')